In [1]:
import pandas as pd

data = pd.read_csv("../../data/twitter16_dataset_with_tvt.csv", lineterminator="\n")
print(data.shape)
data.head()

(818, 7)


,tweet_id,tweet_text,label,tvt,cv_fold,tt,tvt2
0,656955120626880512,correct predictions in back to the future ii U...,false,training,1,training,validation
1,615689290706595840,.@whitehouse in rainbow colors for #scotusmarr...,true,training,3,training,training
2,613404935003217920,cops bought the alleged church shooter burger ...,false,training,2,test,validation
3,731166399389962242,🔥ca kkk grand wizard 🔥 endorses @hillaryclinto...,unverified,training,3,test,training
4,714598641827246081,an open letter to trump voters from his top st...,unverified,training,1,test,training


In [2]:
length = data.shape[0]
length

818

In [3]:
import random

# random.seed(33)

train_w = [0 for i in range(675)]
val_w = [1 for i in range(225)]
test_w = [2 for i in range(100)]

weights = train_w + val_w + test_w
print(f"weights : {len(weights)}")

tvt = []
for i in range(length):
    gacha = random.sample(weights, 1)[0]
    if gacha == 0:
        tvt.append("training")
    elif gacha == 1:
        tvt.append("validation")
    else:
        tvt.append("testting")

print(f"Training : {tvt.count('training')} - {round(tvt.count('training')/length, 3)}")
print(f"Validation : {tvt.count('validation')} - {round(tvt.count('validation')/length, 3)}")
print(f"Testing : {tvt.count('testting')} - {round(tvt.count('testting')/length, 3)}")

weights : 1000
Training : 545 - 0.666
Validation : 192 - 0.235
Testing : 81 - 0.099


In [4]:
data['tvt2'] = pd.Series(tvt)
data.head()

,tweet_id,tweet_text,label,tvt,cv_fold,tt,tvt2
0,656955120626880512,correct predictions in back to the future ii U...,false,training,1,training,validation
1,615689290706595840,.@whitehouse in rainbow colors for #scotusmarr...,true,training,3,training,training
2,613404935003217920,cops bought the alleged church shooter burger ...,false,training,2,test,training
3,731166399389962242,🔥ca kkk grand wizard 🔥 endorses @hillaryclinto...,unverified,training,3,test,training
4,714598641827246081,an open letter to trump voters from his top st...,unverified,training,1,test,training


In [5]:
label_count = data['label'].value_counts()
label_count

true          207
false         205
non-rumor     205
unverified    201
Name: label, dtype: int64

In [6]:
combination = data.apply(lambda row: f"{row['label']}_{row['tvt2']}", axis=1).value_counts()
comparison = {}
for k, comb in combination.items():
    cv_fold = k.split("_")[1]
    label = k.split("_")[0]

    if cv_fold not in comparison:
        comparison[cv_fold] = {}
    
    comparison[cv_fold][label] = comb

labels = data['label'].unique().tolist()

def label_ratio(label_dict, labels):
    total = sum([v for k, v in label_dict.items()])
    
    report = ""
    for l in labels:
        report += f"{round(label_dict[l]/total, 2)}\t"
    
    return report[:-1]

def label_raw_value(label_dict, labels):
    total = sum([v for k, v in label_dict.items()])
    
    report = ""
    for l in labels:
        report += f"{label_dict[l]}\t"
    
    return report[:-1]

labels_str = ','.join(labels)
print(f"\nLabel,{labels_str}")
print(f"Original\t{label_ratio(label_count, labels)}")
for cv, comp in comparison.items():
    print(f"{cv.title()}\t{label_ratio(comp, labels)}")
    
labels_str = ','.join(labels)
print(f"\nLabel,{labels_str}")
print(f"Original\t{label_raw_value(label_count, labels)}")
for cv, comp in comparison.items():
    print(f"{cv.title()}\t{label_raw_value(comp, labels)}")


Label,false,true,unverified,non-rumor
Original	0.25	0.25	0.25	0.25
Training	0.25	0.24	0.25	0.26
Validation	0.24	0.29	0.24	0.23
Testting	0.26	0.25	0.25	0.25

Label,false,true,unverified,non-rumor
Original	205	207	201	205
Training	138	132	135	140
Validation	46	55	46	45
Testting	21	20	20	20


In [8]:
data.to_csv("../../data/twitter16_dataset_with_tvt.csv", index=False)